# Εργασία 2 (Τεχνικές Εξόρυξης Δεδομένων)
## Data Mining: Assignment 2
***
### Μαρία Φριτζελά 1115201400218
***

In [65]:
import glob
import csv
import re
import pandas as pd
from itertools import chain
from IPython.core.display import display
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split

Κατηγοριοποίηση δεδομένων κειμένου από ειδησιογραφικά άρθρα 5 κατηγοριών:</br>
Classifying text data from articles of 5 different categories:

In [2]:
categories = ['business', 'entertainment', 'politics', 'sport', 'tech']

### Get names of files for testing and training
Data set consists of 2225 documents from a news website
corresponding to stories in five topical areas from 2004-2005.</br>
</br>
80% of data points (files) will be used for training, the remaining 20% will be used for testing.</br>
We will be collecting the files names as elements in two lists (one for each purpose)

In [3]:
files_path = 'fulltext/data/'
train_files = []    # list of files names that will be used for training
test_files = []     # list of files names that will be used for testing

for category in categories:
    # get all txt files names from current category
    files = glob.glob(files_path+category+'/*.txt')
    # sort them alphabetically
    files.sort()
    # separate list so that first 80% will be copied into the train_files list
    sep_index = round(len(files) * 0.8)
    train_files.extend(files[:sep_index])
    test_files.extend(files[sep_index:])

Result:

In [ ]:
print("Total files: 2225")
print("# of train files: "+str(len(train_files)))
print("# of test files: "+str(len(test_files)))

### Create file train_set.tsv
Columns: id, title, content, category <br>
_The id is the name of the text file with the first letter of the category prepended (ex. "b001")_

In [ ]:
with open('train_set.tsv', 'wt') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    # write header row
    tsv_writer.writerow(['id', 'title', 'content', 'category'])
    # write rows for all other files
    for file_path in train_files:
        with open(file_path) as f:
            # read all lines of file split them in a list removing '\n'
            # iterate through elements in list removing empty ones (empty strings are False)
            try:
                content = [line for line in f.read().splitlines() if line.strip()]
                # unpack the information we need from the file path
                _, _, cat, name, _ = file_path.replace('.', '/').split('/')
                # Write row
                tsv_writer.writerow([cat[0]+name,
                                 content.pop(0),
                                 " ".join(content),
                                 cat])
            except UnicodeDecodeError:
                print("UnicodeDecodeError for file: "+file_path+". File skipped")


### Create file test_set.tsv
Columns: id, title, content

In [ ]:
with open('test_set.tsv', 'wt') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    # write header row
    tsv_writer.writerow(['id', 'title', 'content'])
    # write rows for all other files
    for file_path in test_files:
        with open(file_path) as f:
            # read all lines of file split them in a list removing '\n'
            # iterate through elements in list removing empty ones (empty strings are False)
            try:
                content = [line for line in f.read().splitlines() if line.strip()]
                # unpack the information we need from the file path
                _, _, cat, name, _ = file_path.replace('.', '/').split('/')
                # Write row
                tsv_writer.writerow([cat[0]+name,
                                 content.pop(0),
                                 " ".join(content)])
            except UnicodeDecodeError:
                print("UnicodeDecodeError for file: "+file_path+". File skipped")

### Create a file with the full dataset: dataset.tsv


In [4]:
with open('dataset.tsv', 'wt') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    # write header row
    tsv_writer.writerow(['id', 'title', 'content', 'category'])
    # write rows for all other files
    for file_path in chain(train_files, test_files):
        with open(file_path) as f:
            # read all lines of file split them in a list removing '\n'
            # iterate through elements in list removing empty ones (empty strings are False)
            try:
                content = [line for line in f.read().splitlines() if line.strip()]
                # unpack the information we need from the file path
                _, _, cat, name, _ = file_path.replace('.', '/').split('/')
                # Write row
                tsv_writer.writerow([cat[0]+name,
                                 content.pop(0),
                                 " ".join(content),
                                 cat])
            except UnicodeDecodeError:
                print("UnicodeDecodeError for file: "+file_path+". File skipped")

UnicodeDecodeError for file: fulltext/data/sport/199.txt. File skipped


Create a DataFrame for the data_set (id column as the index)

In [5]:
#testdf = pd.read_csv("test_set.tsv", sep='\t', index_col='id')
#traindf = pd.read_csv("train_set.tsv", sep='\t', index_col='id')
datadf = pd.read_csv("dataset.tsv", sep='\t', index_col='id')
datadf

,title,content,category
id,,,
b001,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarner...,business
b002,Dollar gains on Greenspan speech,The dollar has hit its highest level against t...,business
b003,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yuko...,business
b004,High fuel prices hit BA's profits,British Airways has blamed high fuel prices fo...,business
b005,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Domec...,business
...,...,...,...
t397,BT program to beat dialler scams,BT is introducing two initiatives to help beat...,tech
t398,Spam e-mails tempt net shoppers,Computer users across the world continue to ig...,tech
t399,Be careful how you code,A new European directive could put software wr...,tech


## 1 Δημιουργία WordCloud
**Create a WordCloud for the articles of each category**

In [7]:
# Create our own stopWord list:
stopwords = set(STOPWORDS)
stopwords.update(['say', 'said', 'saying', 'will', 'many', 'new', 'people', 'now', 'one'])

### Business

In [ ]:
#create wordcloud
# select rows where the id contains 'b' (=business) using filter
wordcloud_business = WordCloud(
    width = 800,
    height = 800,
    background_color = 'black',
    stopwords = stopwords).generate(" ".join(title+' '+content
                                             for title, content in datadf.filter(like='b', axis=0)
                                                                   [['title', 'content']].values.tolist()))

In [ ]:
business_image = wordcloud_business.to_image()
display(business_image)

### Entertainment

In [ ]:
wordcloud_entertainment = WordCloud(
    width = 800,
    height = 800,
    background_color = 'black',
    stopwords = stopwords).generate(str(datadf.filter(like='e', axis=0)[['title', 'content']].values))

In [ ]:
#show it
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud_entertainment)
plt.axis("off")
plt.tight_layout(pad = 0)
plt.show()

### Politics

In [ ]:
wordcloud_politics = WordCloud(
    width = 800,
    height = 800,
    background_color = 'black',
    stopwords = stopwords).generate(str(datadf.filter(like='p', axis=0)[['title', 'content']].values))

In [ ]:
#show it
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud_politics)
plt.axis("off")
plt.tight_layout(pad = 0)
plt.show()

### Sport

In [ ]:
wordcloud_sport = WordCloud(
    width = 800,
    height = 800,
    background_color = 'black',
    stopwords = stopwords).generate(str(datadf.filter(like='s', axis=0)[['title', 'content']].values))

In [ ]:
#show it
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud_sport)
plt.axis("off")
plt.tight_layout(pad = 0)
plt.show()

### Tech

In [ ]:
wordcloud_tech = WordCloud(
    width = 800,
    height = 800,
    background_color = 'black',
    stopwords = stopwords).generate(str(datadf.filter(like='t', axis=0)[['title', 'content']].values))

In [ ]:
#show it
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud_tech)
plt.axis("off")
plt.tight_layout(pad = 0)
plt.show()

## 2 Υλοποίηση Κατηγοριοποίησης (Classification)
**Data Classification**

### A) Cleaning and Pre-processing the data
Create a Pandas Series by concatenating the title and content column of the datadf <br>
Clean up text:<br>
- Add a space before performing the sum to not connect words together accidentally
- Make all the words lower case to facilitate clean up, using `.lower`
- Remove our list of stopwords
- Remove punctuation and special characters using `re.sub`
- Remove all words containing digits, and any digits using `re.sub`.

In [62]:
datadf['text'] = datadf[['title', 'content']]\
    .apply(lambda row: ' '.join(row.values.astype(str)), axis=1)\
    .apply(lambda item: list(filter(lambda word: word not in stopwords, item.lower().split())))\
    .apply(lambda item: re.sub('[^A-Za-z0-9 ]+', '', ' '.join(item)))\
    .apply(lambda item: re.sub(r'\w*\d\w*', '', item))

Seperate data into train (80%) and test (20%) set

In [77]:
X_train, X_test, y_train, y_test = train_test_split(datadf.text, datadf.category, test_size=0.2)

#### Bag-of-words
Create bag-of-words vector

In [78]:
bow_vectorizer = CountVectorizer(max_features=3000, stop_words='english')

bow_X = bow_vectorizer.fit_transform(X_train)


In [79]:
pd.DataFrame(bow_X[0:1].T.todense(), index=bow_vectorizer.get_feature_names(), columns=["counts"])\
.sort_values(by=["counts"],ascending=False)

,counts
iraq,6
mrs,3
care,3
war,2
international,2
...,...
fees,0
fell,0
fellow,0
felt,0


#### TF-IDF

In [80]:
tfidf_vectorizer= TfidfVectorizer(ngram_range=(1,2), max_features=3000, stop_words='english')

tfidf = tfidf_vectorizer.fit_transform(X_train)

In [81]:
pd.DataFrame(tfidf[0:1].T.todense(), index=tfidf_vectorizer.get_feature_names(), columns=["tfidf"])\
.sort_values(by=["tfidf"],ascending=False)


,tfidf
iraq,0.532979
mrs,0.357154
care,0.274610
operations,0.184546
aid,0.177660
...,...
file,0.000000
filed,0.000000
files,0.000000
filesharing,0.000000


### B) Classification
